In [65]:
import requests
from bs4 import BeautifulSoup
import csv
startYear = 2015
endYear = 2018
totalWeeks=17
weekL = range(1,totalWeeks+1)
yearL = range(startYear,endYear+1)
categoryL = ['Passing','Rushing','Receiving','Placekick','Defensive','Punting']

for wk in weekL:
    for yr in yearL:
        for cat in categoryL:
            print('cat',cat,'yr',yr,'wk',wk)
            #create the url
            url = 'http://www.nfl.com/stats/weeklyleaders?week={week}&season={year}&type=REG&showCategory={category}'.format(
                week=str(wk),year=str(yr),category=cat)
            #Pull the html code
            page = requests.get(url)
            soup = BeautifulSoup(page.text,'html.parser')
            
            #extract the table
            tableSoup = soup.find('tbody')
            headerSoup = soup.find_all('th')
            
            #get the data out of the table
            data = pullDataTable(tableSoup,headerSoup)
            
            #Write the data to a csv file
            
            with open('stats/{category}_yr{year}_wk{week}.csv'.format(category=cat,year=str(yr),week=str(wk)),'a') as csv_file:
                writer = csv.writer(csv_file)
                for x in range(len(data)):
                    writer.writerow(data[x])


cat Passing yr 2015 wk 1
cat Rushing yr 2015 wk 1
cat Receiving yr 2015 wk 1
cat Placekick yr 2015 wk 1
cat Defensive yr 2015 wk 1
cat Punting yr 2015 wk 1
cat Passing yr 2016 wk 1
cat Rushing yr 2016 wk 1
cat Receiving yr 2016 wk 1
cat Placekick yr 2016 wk 1
cat Defensive yr 2016 wk 1
cat Punting yr 2016 wk 1
cat Passing yr 2017 wk 1
cat Rushing yr 2017 wk 1
cat Receiving yr 2017 wk 1
cat Placekick yr 2017 wk 1
cat Defensive yr 2017 wk 1
cat Punting yr 2017 wk 1
cat Passing yr 2018 wk 1
cat Rushing yr 2018 wk 1
cat Receiving yr 2018 wk 1
cat Placekick yr 2018 wk 1
cat Defensive yr 2018 wk 1
cat Punting yr 2018 wk 1
cat Passing yr 2015 wk 2
cat Rushing yr 2015 wk 2
cat Receiving yr 2015 wk 2
cat Placekick yr 2015 wk 2
cat Defensive yr 2015 wk 2
cat Punting yr 2015 wk 2
cat Passing yr 2016 wk 2
cat Rushing yr 2016 wk 2
cat Receiving yr 2016 wk 2
cat Placekick yr 2016 wk 2
cat Defensive yr 2016 wk 2
cat Punting yr 2016 wk 2
cat Passing yr 2017 wk 2
cat Rushing yr 2017 wk 2
cat Receiving 

cat Passing yr 2015 wk 14
cat Rushing yr 2015 wk 14
cat Receiving yr 2015 wk 14
cat Placekick yr 2015 wk 14
cat Defensive yr 2015 wk 14
cat Punting yr 2015 wk 14
cat Passing yr 2016 wk 14
cat Rushing yr 2016 wk 14
cat Receiving yr 2016 wk 14
cat Placekick yr 2016 wk 14
cat Defensive yr 2016 wk 14
cat Punting yr 2016 wk 14
cat Passing yr 2017 wk 14
cat Rushing yr 2017 wk 14
cat Receiving yr 2017 wk 14
cat Placekick yr 2017 wk 14
cat Defensive yr 2017 wk 14
cat Punting yr 2017 wk 14
cat Passing yr 2018 wk 14
cat Rushing yr 2018 wk 14
cat Receiving yr 2018 wk 14
cat Placekick yr 2018 wk 14
cat Defensive yr 2018 wk 14
cat Punting yr 2018 wk 14
cat Passing yr 2015 wk 15
cat Rushing yr 2015 wk 15
cat Receiving yr 2015 wk 15
cat Placekick yr 2015 wk 15
cat Defensive yr 2015 wk 15
cat Punting yr 2015 wk 15
cat Passing yr 2016 wk 15
cat Rushing yr 2016 wk 15
cat Receiving yr 2016 wk 15
cat Placekick yr 2016 wk 15
cat Defensive yr 2016 wk 15
cat Punting yr 2016 wk 15
cat Passing yr 2017 wk 15
ca

In [57]:
def readBodyLinks(linkSoup):
    """Reads in the soup of a single column link from a row. extracts all of the strings and joins it together with white
    space """
    contentL = linkSoup.contents
    parts=[]
    for i in range(len(contentL)):
        try:
            parts.append(contentL[i].strip())
        except:
            #maybe we need to extract the string because it's caught up in soup?
            if contentL[i].contents==[]:
                parts.append('')
            else:
                stringPart = contentL[i].contents[0]
                parts.append(stringPart.strip())
    return ' '.join(parts).strip()

In [15]:
def pullDataTable(tableSoup,headerSoupL):
    """Takes the soup from the header (find_all(th)) and the table (find(tbody)) and then returns a list of lists.
    the first row will be the title of each column, and then each subsequent row will be individual players"""
    #Deal with the header first. We need to learn how the data is organized
    #We should keep track of which columns have links in the header. Those should just be numbers in the body
    linkFlag = []
    header = []
    #move through columns of the header
    for j in range(len(headerSoupL)):
        linkL = headerSoupL[j].find_all('a')
        if len(linkL)>0:
            linkFlag.append(True)
            colTitle = linkL[0].contents[0]
        else:
            linkFlag.append(False)
            colTitle = headerSoup[j].contents[0]
        header.append(colTitle)
    data = [header]
    #start reading through rows
    rowSoupL = tableSoup.find_all('tr')
    for i in range(len(rowSoupL)):
        rowData = []
        colSoupL = rowSoupL[i].find_all('td')
        for j in range(len(colSoupL)):
            if linkFlag[j]:
                #If the header was a link, the the body probably won't be
                numStr = colSoupL[j].contents[0]
                if '.' in numStr:
                    rowData.append(float(colSoupL[j].contents[0]))
                else:
                    rowData.append(int(colSoupL[j].contents[0]))
            else:
                #The header wasn't a link, so the link is probably in the body? There's an exception that we need to look
                # out for... if header[j]='Opp', then the contents will also contain 'home' or 'away' data....
                rowData.append(readBodyLinks(colSoupL[j]))
        data.append(rowData)
    return data
                

In [61]:
print(url)

http://www.nfl.com/stats/weeklyleaders?week=1&season=2010&type=REG&showCategory=Kicking


In [6]:
import requests
from bs4 import BeautifulSoup
import csv

url = "http://www.nfl.com/stats/weeklyleaders?week=1&season=2018&showCategory=Passing"
nextURL = 'http://www.nfl.com/stats/weeklyleaders?week=1&season=2018&type=REG&showCategory=Passing'
page = requests.get(url)

soup = BeautifulSoup(page.text, 'html.parser')
print(soup)


<!DOCTYPE html>

<!--[if lt IE 7 ]> <html lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:og="http://ogp.me/ns#" xmlns:fb="http://www.facebook.com/2008/fbml" class="ie ie6"> <![endif]-->
<!--[if IE 7 ]>    <html lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:og="http://ogp.me/ns#" xmlns:fb="http://www.facebook.com/2008/fbml" class="ie ie7"> <![endif]-->
<!--[if IE 8 ]>    <html lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:og="http://ogp.me/ns#" xmlns:fb="http://www.facebook.com/2008/fbml" class="ie ie8"> <![endif]-->
<!--[if IE 9 ]>    <html lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:og="http://ogp.me/ns#" xmlns:fb="http://www.facebook.com/2008/fbml" class="ie ie9"> <![endif]-->
<!--[if (gt IE 9)|!(IE)]><!--> <html lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#"> <!--<![endif]-->
<head>
<script>
    window['adrum-start-time'] = new Date().getTime();
    window["adrum-app-key"] = "AD-A

In [7]:
linkSoupList = soup.find_all('a')
gameList = []
for i in range(len(linkSoupList)):
    urlParts = linkSoupList[i]['href'].split('/')
    if urlParts[1]=='gamecenter':
        gameList.append(linkSoupList[i])
print(gameList[0].contents)


['40-48']


In [8]:
tableSoup = soup.find('tbody')
#print(tableSoup)
print(tableSoup.find_all('tr')[0].find_all('td')[3].contents)
content = tableSoup.find_all('tr')[0].find_all('td')[0].contents
parts = []
for i in range(len(content)):
    try:
        #maybe it's a string?
        parts.append(content[i].strip())
    except:
        #maybe we need to extract the string because it's caught up in soup?
        stringPart = content[i].contents[0]
        parts.append(stringPart.strip())
print(' '.join(parts))


['\n', <span class="red">L</span>, '\r\n\t\r\n\r\n\xa0', <a href="/gamecenter/2018090906/2018/REG1/buccaneers@saints">40-48</a>, '\n']
 Drew Brees 


In [13]:
headerSoup = soup.find_all('th')
#for i in range(len(headerSoup)):
print(headerSoup[2].contents[0])

Opp


In [16]:
data = pullDataTable(tableSoup,headerSoup)

for i in range(len(data)):
    print(data[i])

['Name', 'Team', 'Opp', 'Score', 'Comp', 'Att', 'Yds', 'TD', 'Int', 'Sck', 'FUM', 'Rate']
['Drew Brees', 'NO', 'vs TB', 'L  40-48', 37, 45, 439, 3, 0, 1, 0, 129.5]
['Philip Rivers', 'LAC', 'vs KC', 'L  28-38', 34, 51, 424, 3, 1, 1, 0, 103.7]
['Ryan Fitzpatrick', 'TB', '@ NO', 'W  48-40', 21, 28, 417, 4, 0, 0, 0, 156.2]
['Ben Roethlisberger', 'PIT', '@ CLE', 'T 21-21', 23, 41, 335, 1, 3, 4, 2, 60.5]
['Case Keenum', 'DEN', 'vs SEA', 'W  27-24', 25, 39, 329, 3, 3, 1, 0, 84.2]
['Andrew Luck', 'IND', 'vs CIN', 'L  23-34', 39, 53, 319, 2, 1, 2, 0, 93.2]
['Derek Carr', 'OAK', 'vs LA', 'L  13-33', 29, 40, 303, 0, 3, 1, 0, 62.8]
['Russell Wilson', 'SEA', '@ DEN', 'L  24-27', 19, 33, 298, 3, 2, 6, 0, 92.7]
['Aaron Rodgers', 'GB', 'vs CHI', 'W  24-23', 20, 30, 286, 3, 0, 2, 0, 130.7]
['Matthew Stafford', 'DET', 'vs NYJ', 'L  17-48', 27, 46, 286, 1, 4, 0, 0, 47.9]
['Tom Brady', 'NE', 'vs HOU', 'W  27-20', 26, 39, 277, 3, 1, 2, 0, 102.2]
['Jimmy Garoppolo', 'SF', '@ MIN', 'L  16-24', 15, 33, 261, 1